# Multi-Output Learning: Regressor Chains

<table>
  <thead>
    <tr>
      <th>Step</th>
      <th>Input Data</th>
      <th>Predicted Target(s)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>Base Features</td>
      <td>Tensile Strength</td>
    </tr>
    <tr>
      <td>2</td>
      <td>Base Features + Tensile Strength</td>
      <td>Flexural Strength</td>
    </tr>
    <tr>
      <td>3</td>
      <td>Base Features + Tensile Strength + Flexural Strength</td>
      <td>Elongation at Break<br>Impact Strength</td>
    </tr>
  </tbody>
</table>

In [ ]:
from ml_tools.ML_chain import DragonChainOrchestrator, derive_next_step_schema
from ml_tools.ML_inference import DragonInferenceHandler
from ml_tools.ML_utilities import DragonArtifactFinder
from ml_tools.ML_models import DragonNodeModel, DragonGateModel
from ml_tools.data_exploration import summarize_dataframe
from ml_tools.utilities import load_dataframe_greedy, save_dataframe_with_schema
from ml_tools.schema import FeatureSchema

from paths import PM
from helpers.constants import TARGET_tensile_strength, TARGET_flexural_strength, TARGET_elongation_at_break, TARGET_impact_strength

## Init

In [ ]:
imputed_df = load_dataframe_greedy(directory=PM.mice_datasets)

In [ ]:
chain_orchestrator = DragonChainOrchestrator(initial_dataset=imputed_df,
                                             all_targets=[TARGET_tensile_strength, TARGET_flexural_strength, TARGET_elongation_at_break, TARGET_impact_strength])

In [ ]:
# Helper function
def update_chain_orchestrator(artifacts_directory, model_class, previous_schema: FeatureSchema, device="cuda"):
    finder = DragonArtifactFinder(directory=artifacts_directory, load_scaler=True, load_schema=False)
    
    inference_model = model_class.load_architecture(finder.model_architecture_path)
    
    handler = DragonInferenceHandler(model=inference_model,
                                    state_dict=finder.weights_path, # type: ignore
                                    scaler=finder.scaler_path,
                                    device=device)
    
    # update
    chain_orchestrator.update_with_inference(handler=handler)
    # return new schema
    new_schema = derive_next_step_schema(current_schema=previous_schema, handler=handler)
    return new_schema

## Step 1 - Tensile Strength

In [ ]:
feature_schema_1 = FeatureSchema.from_json(PM.engineering_artifacts)

In [ ]:
df_1 = chain_orchestrator.get_training_data(target_subset=[TARGET_tensile_strength])

In [ ]:
summarize_dataframe(df_1)

Save training dataset validated with schema

In [ ]:
save_dataframe_with_schema(df=df_1, full_path=PM.step1_data_file, schema=feature_schema_1)

## Step 2 - Flexural Strength

Update chain orchestrator

In [ ]:
feature_schema_2 = update_chain_orchestrator(artifacts_directory=PM.train_artifacts_1, model_class=DragonNodeModel, previous_schema=feature_schema_1)

In [ ]:
df_2 = chain_orchestrator.get_training_data(target_subset=[TARGET_flexural_strength])

In [ ]:
summarize_dataframe(df_2)

Save training dataset validated with schema

In [ ]:
feature_schema_2.to_json(PM.engineering_artifacts_2)
feature_schema_2.save_artifacts(PM.engineering_artifacts_2)

In [ ]:
save_dataframe_with_schema(df=df_2, full_path=PM.step2_data_file, schema=feature_schema_2)

## Step 3 - Elongation at Break & Impact Strength

Update chain orchestrator

In [ ]:
feature_schema_3 = update_chain_orchestrator(artifacts_directory=PM.train_artifacts_2, model_class=DragonGateModel, previous_schema=feature_schema_2)

In [ ]:
df_3 = chain_orchestrator.get_training_data(target_subset=[TARGET_elongation_at_break, TARGET_impact_strength], dropna_how="any")

In [ ]:
summarize_dataframe(df_3)

Save training dataset validated with schema

In [ ]:
feature_schema_3.to_json(PM.engineering_artifacts_3)
feature_schema_3.save_artifacts(PM.engineering_artifacts_3)

In [ ]:
save_dataframe_with_schema(df=df_3, full_path=PM.step3_data_file, schema=feature_schema_3)